<a href="https://colab.research.google.com/github/snastase/isc-tutorial/blob/master/isc_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Intersubject correlation (ISC) tutorial</h1>
This tutorial jupyter notebook accompanies the manuscript "Measuring shared responses across subjects using intersubject correlation" by Nastase, Gazzola, Hasson, and Keysers. The goal of the tutorial is to introduce basic intersubject correlation (ISC) analyses ([Hasson et al., 2004](https://doi.org/10.1126/science.1089506), [2010](https://doi.org/10.1016/j.tics.2009.10.011)) and subsequent statistical tests as implemented in Python using the Brain Imaging Analysis Kit ([BrainIAK](http://brainiak.org/)). Click *Open in playground* to interactively run and edit cells (you may need to sign into a Google account), and use *File* > *Save a copy in Drive...* or *Save a copy in GitHub...* to save your changes. To execute a code cell, click in that cell and press _Shift_ + _Enter_ (or _Shift_ + _Return_ on a Mac). The first time you run a code cell in playground mode, you may receive a warning and a prompt to reset runtimes; click _Run anyway_ followed by _Yes_.

---

Author: Samuel A. Nastase

## Getting started
First, we'll need to [install BrainIAK](http://brainiak.org/docs/installation.html) and its requirements in the cloud instance—this may take a few minutes. For this tutorial, we'll install BrainIAK directly from the [GitHub repository](https://github.com/brainiak/brainiak) to get the most recent features. Note that if you are running this notebook locally, the installation process may vary and you should not execute this code cell. If you want to install BrainIAK locally, run `pip install git+https://github.com/snastase/brainiak.git` in a separate code cell or follow the online [installation instructions](http://brainiak.org/docs/installation.html).

In [0]:
# Install BrainIAK requirements in Google Colab Linux cloud instance
!apt install build-essential libgomp1 libmpich-dev mpich python3-dev \
             python3-pip python3-venv

# Install most recent BrainIAK from GitHub in the cloud instance
!pip install pip==9.0.1
!pip install git+https://github.com/brainiak/brainiak.git

Next, we'll import the relevant functions from BrainIAK

In [0]:
from brainiak.isc import (isc, isfc, bootstrap_isc, permutation_isc,
                          timeshift_isc, phaseshift_isc,
                          compute_summary_statistic)
from brainiak.io import load_boolean_mask, load_images
from brainiak.image import mask_images, MaskedMultiSubjectData

If unable to install BrainIAK, we can use basic ISC functionality without the full BrainIAK package. We'll download `isc_standalone.py` from the [GitHub repository](https://github.com/snastase/isc-tutorial) for this tutorial and load the necessary modules locally. If you've cloned the `isc-tutorial` GitHub repository locally, this step is not necessary (as the local directory for the repository already contains `isc_standalone.py`).

In [0]:
# Download the standalone module if not using BrainIAK
from urllib.request import urlretrieve
urlretrieve('https://github.com/snastase/isc-tutorial/'
            'raw/master/isc_standalone.py', 'isc_standalone.py');

If you're using the `isc_standalone.py` module instead of BrainIAK, import the relevant functions from `isc_standalone`.

In [0]:
from isc_standalone import (isc, isfc, bootstrap_isc, permutation_isc,
                            timeshift_isc, phaseshift_isc,
                            compute_summary_statistic, load_images,
                            load_boolean_mask, mask_images,
                            MaskedMultiSubjectData)

Finally, we'll load several other useful Python modules.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, pearsonr, zscore
from scipy.spatial.distance import squareform
from statsmodels.stats.multitest import multipletests
import nibabel as nib

## Example data
We'll create a simple simulated dataset for quickly applying ISC analyses, then later apply the analyses to a real fMRI dataset where participants listened to a spoken narrative ([Pie Man](https://themoth.org/stories/pie-man) by Jim O'Grady). Our simulated data will have 1,000 voxels in total comprising 10 "networks" and 300 time points (or TRs).

In [0]:
# Set parameters for toy time series data
n_subjects = 20
n_TRs = 300
n_voxels = 1000

# Create simple simulated data with high intersubject correlation
def simulated_timeseries(n_subjects, n_TRs, n_voxels=1, noise=1):
  signal = np.random.randn(n_TRs, n_voxels // 100)
  data = [zscore(np.repeat(signal, 100, axis=1) +
                 np.random.randn(n_TRs, n_voxels) * noise,
                 axis=0)
          for subject in np.arange(n_subjects)]
  return data

# List of subject datasets
data = simulated_timeseries(n_subjects, n_TRs, n_voxels=n_voxels)

In [0]:
# Inspect the shape of one of our simulated datasets
print(f"Simulated data shape first subject: {data[0].shape} "
      f"\ni.e., {data[0].shape[0]} time points and {data[0].shape[1]} voxels")

# Create a simple visualization of the data
plt.matshow(data[0], cmap='RdYlBu_r', vmin=-3, vmax=3)
plt.grid(False)
plt.xlabel('voxels')
plt.ylabel('time points');

## ISC analysis
Let's start very simple by computing the ISC for a single voxel (or ROI) across only two participants. This should give us a simple Pearson correlation value (and should match other implementations of Pearson correlation). Note that when you call the `isc` function with `verbose=True` (the default), it outputs some warnings describing what it infers about the input data. If these don't match your assumptions, your input data may be organized improperly.

In [0]:
# Get the time series for a single voxel in two subjects
subject_a = data[0][:, 0]
subject_b = data[1][:, 0]

# Check the shape of these mini-datasets
print(f"Subject A, first voxel, shape = {subject_a.shape} "
      f"\nSubject B, first voxel, shape = {subject_b.shape}")

# Combine these into a list
both_subjects = [subject_a, subject_b]

# Compute the ISC for this voxel across the two subjects
iscs = isc(both_subjects, pairwise=True)
print(f"ISC for first voxel across subjects A and B = {iscs[0]}")

# NB: even for a single voxel, the output ISC is shaped to 
# to accommodate an n_ISCs x n_voxels matrix
print(f"ISC output shape = {iscs.shape}"
      f"\ni.e., {iscs.shape[0]} ISC value(s) by {iscs.shape[0]} voxel(s)")

# Check that ISC output matches of other correlation functions in python
numpy_corrcoef = np.corrcoef(subject_a, subject_b)[0, 1]

scipy_pearsonr = pearsonr(subject_a, subject_b)[0]

print(f"BrainIAK ISC = {iscs[0]:.6f}"
      f"\nNumpy's correlation = {numpy_corrcoef:.6f}"
      f"\nScipy's correlation = {scipy_pearsonr:.6f}")
assert np.isclose(iscs, numpy_corrcoef) and np.isclose(iscs, scipy_pearsonr)

BrainIAK uses Python's logging functionality. To see non-critical messages while running ISC analyses, we temporarily can set the logging level to 'INFO'.

In [0]:
# Import logging module and set level to INFO
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

# Re-run the previous ISC analyses to see logged info
iscs = isc(both_subjects, pairwise=True)

In [0]:
# Set logging back to default level of WARNING
logging.getLogger().setLevel(logging.WARNING)

When there are three or more subjects, we can compute ISCs using either the pairwise approach (`pairwise=True`), where we compute ISCs between each pair of subjects, or the leave-one-out (`pairwise=False`) approach, where we compute ISCs between each subject and the average time series of other subjects.

### Pairwise approach
Now we'll run the full-scale ISC analysis across all voxels and subjects using the pairwise approach. For a given voxel, the correlations between each pair of subjects are represented in a vector of length
```
n_subjects * (n_subjects - 1) / 2
```
or 190 pairs for 20 subjects. This vector of pairs corresponds to the off-diagonal values of a symmetric subjects-by-subjects correlation matrix.



In [0]:
# Pairwise approach across all subjects and voxels
iscs = isc(data, pairwise=True)

# Check shape of output ISC values
print(f"ISC values shape = {iscs.shape} \ni.e., {iscs.shape[0]} "
      f"pairs and {iscs.shape[1]} voxels"
      f"\nMinimum ISC = {np.amin(iscs):.3f}; "
      f"maximum ISC = {np.amax(iscs):.3f}")

For a given voxel, we can convert the vector of pairs to the full correlation matrix for visualization. In the simulated dataset, all subjects were designed to have high ISCs; however, we can add noise to some of the subjects and then visualize the ISC matrix.

In [0]:
# Visualize the correlation matrix for one voxel
isc_matrix = squareform(iscs[:, 0])
np.fill_diagonal(isc_matrix, 1)
sns.heatmap(isc_matrix, cmap="RdYlBu_r", vmin=-1, vmax=1, square=True, 
            xticklabels=range(1, 21), yticklabels=range(1, 21))
plt.xlabel('subjects')
plt.ylabel('subjects')
plt.show()

# Create noisier data
noisy_data = np.dstack((np.dstack((
    simulated_timeseries(n_subjects // 2, n_TRs,
                         n_voxels=n_voxels, noise=1))),
                        np.dstack((
    simulated_timeseries(n_subjects // 2, n_TRs,
                         n_voxels=n_voxels, noise=5)))))

# Recompute ISC and visualize data with noisy subjects
noisy_iscs = isc(noisy_data, pairwise=True)
isc_matrix = squareform(noisy_iscs[:, 0])
np.fill_diagonal(isc_matrix, 1)
sns.heatmap(isc_matrix, cmap="RdYlBu_r", vmin=-1, vmax=1, square=True, 
            xticklabels=range(1, 21), yticklabels=range(1, 21))
plt.xlabel('subjects')
plt.ylabel('subjects')
plt.show()

### Leave-one-out approach
Instead of computing ISCs between each pair of subjects, for each subject we can compute the ISC between that subject and the average of all other subjects. Notice that the observed ISC values are typically higher in the leave-one-out approach due to computing correlations between the left-out subject and the cleaner averaged time series from the remaining subjects.

In [0]:
# Leave-one-out approach
iscs = isc(data, pairwise=False)

# Check shape of output ISC values
print(f"ISC values shape = {iscs.shape} \ni.e., {iscs.shape[0]} "
      f"left-out subjects and {iscs.shape[1]} voxel(s)"
      f"\nMinimum ISC = {np.amin(iscs):.3f}; "
      f"maximum ISC = {np.amax(iscs):.3f}")

### Input types
Currently, we're a submitting a list of numpy arrays to BrainIAK's `isc` function where each item in the list is a subject's response time course over some number of voxels. Alternatively, we could stack subjects along the 3rd dimension (`np.dstack`) into a single 3-dimensional numpy array and submit this to the `isc` function. If the `isc` function receives a single numpy array, it will assume that the last dimension indexes subjects.

In [0]:
# Input a list of subjects (same as before)
iscs = isc(data, pairwise=False)

# Stack subjects in 3rd-dimension and recompute ISC
data_stack = np.dstack(data)
print(f"Stacked data shape = {data_stack.shape}"
      f"\ni.e., {data_stack.shape[0]} time points, {data_stack.shape[1]} "
      f"voxels, and {data_stack.shape[2]} subjects")

# Input stacked numpy array
iscs_from_stack = isc(data_stack, pairwise=False)

# Make sure the ISC outputs are the same
assert np.array_equal(iscs, iscs_from_stack)

### Summary statistics
Rather than returning ISC values for each pair of subject (in the pairwise approach) or each left-out subject (in the leave-one-out approach), we can use the `summary_statistic` argument to output either the mean or median across the values. Note that by default `summary_statistic=False`. If we request the mean ISC value, the `isc` function will internally apply the Fisher *z*-transformation (`np.arctanh`) prior to computing the mean, then apply the inverse Fisher *z*-transformation (`np.tanh`) to the mean value.

In [0]:
# Compute mean leave-one-out ISC
iscs = isc(data, pairwise=False, summary_statistic='mean')

print(f"ISC values shape = {iscs.shape} \ni.e., the mean value across "
      f"left-out subjects for {iscs.shape[0]} voxel(s)"
      f"\nMean ISC for first voxel = {iscs[0]:.3f}")

# Compute median leave-one-out ISC
iscs = isc(data, pairwise=False, summary_statistic='median')

print(f"ISC values shape = {iscs.shape} \ni.e., the median value across "
      f"left-out subjects for {iscs.shape[0]} voxel(s)"
      f"\nMedian ISC for first voxel = {iscs[0]:.3f}")

## Statistical tests
BrainIAK provides several nonparametric statistical tests for ISC analysis. Nonparametric tests are preferred due to the inherent correlation structure across ISC values—each subject contributes to the ISC of other subjects, violating assumptions of independence required for standard parametric tests (e.g., *t*-test, ANOVA). The nonparametric statistical tests discussed below return the actual observed ISC values, *p*-values, and the resampling distribution (the bootstrap hypothesis test also returns confidence intervals around the observed ISC statistic). For expediency, we only use 200 resampling iterations here, but 1,000 or more iterations are generally recommended.

### Phase randomization
One approach for statistically assessing ISCs is to randomize the phase of time series across subjects prior to computing ISCs (e.g., [Lerner et al., 2011](https://doi.org/10.1523/jneurosci.3684-10.2011); [Simony et al., 2016](https://doi.org/10.1038/ncomms12141)). This method requires recomputing ISC at each iteration of the randomization test, and is therefore slow. In the pairwise approach, we phase-randomize each subject prior to computing ISCs; however, in the leave-one-out approach, we only phase-randomize the left-out subject prior to computing ISC. At each iteration of the phase randomization test, the same random phase shift is used across all voxels to preserve the spatial autocorrelation of typical fMRI data.

In [0]:
# Phase randomization using pairwise approach (takes a couple minutes)
observed, p, distribution = phaseshift_isc(data, pairwise=True,
                                           summary_statistic='median',
                                           n_shifts=200)

In [0]:
# Inspect shape of null distribution
print(f"Null distribution shape = {distribution.shape}"
      f"\ni.e., {distribution.shape[0]} randomizations "
      f"and {distribution.shape[1]} voxels")

# Get actual ISC value and p-value for first voxel
print(f"Actual observed ISC value for first voxel = {observed[0]:.3f},"
      f"\np-value from randomization test = {p[0]:.3f}")

### Circular time-shift randomization
A conceptually similar nonparametric approach is to circularly shift the response time series across subjects by random offsets ([Kauppi et al., 2014](https://doi.org/10.3389/fninf.2014.00002)). Time points that would be shifted beyond the end of the time series are wrapped around to the beginning of the time series.

In [0]:
# Circular time-shift using pairwise approach (takes a couple minutes)
observed, p, distribution = timeshift_isc(data, pairwise=True,
                                          summary_statistic='median',
                                          n_shifts=200)

In [0]:
# Inspect shape of null distribution
print(f"Null distribution shape = {distribution.shape}"
      f"\ni.e., {distribution.shape[0]} randomizations "
      f"and {distribution.shape[1]} voxels")

# Get actual ISC value and p-value for first voxel
print(f"Actual observed ISC value for first voxel = {observed[0]:.3f},"
      f"\np-value from randomization test = {p[0]:.3f}")

### Bootstrap hypothesis test
We can also perform group-level statistical tests that operate directly on the observed ISC values and do not require recomputing ISCs. For one-sample tests, we can resample subjects with replacement to construct a bootstrap distribution around our observed ISC statistic ([Chen et al., 2016](https://doi.org/10.1016/j.neuroimage.2016.05.023)). We can compute confidence intervals around the test statistic using the `ci_percentile` option (default 95%). Hypothesis test is performed by shifting the bootstrap distribution to zero. Note that when constructing the bootstrap distribution using the pairwise approach, subjects (i.e., rows and columns in the subject-by-subject correlation matrix) are sampled with replacement, not pairs (which would disrupt the correlation structure among pairs).

In [0]:
# Compute ISCs and then run bootstrap hypothesis test on ISCs
iscs = isc(data, pairwise=True, summary_statistic=None)
observed, ci, p, distribution = bootstrap_isc(iscs, pairwise=True,
                                              ci_percentile=95,
                                              summary_statistic='median',
                                              n_bootstraps=200)

In [0]:
# Inspect shape of null distribution
print(f"Null distribution shape = {distribution.shape}"
      f"\ni.e., {distribution.shape[0]} bootstraps "
      f"and {distribution.shape[1]} voxels")

# Get actual ISC value and p-value for first voxel
print(f"Actual observed ISC value for first voxel = {observed[0]:.3f},"
      f"\np-value from bootstrap hypothesis test = {p[0]:.3f}")

### Permutation test
We can use a permutation test to statistically evaluate one- or two-sample tests ([Chen et al., 2016](https://doi.org/10.1016/j.neuroimage.2016.05.023)). In the case of a one-sample test, we use a sign-flipping (-1, +1) approach applied to the observed ISC. For a two-sample test, we supply a `group_assignment` list containing the group labels for each subject. The order of the group assignment list must match the order in which the subjects are supplied to the `isc` function. At each iteration, we randomly reassign the group labels, then compute the test statistic. In the one-sample test, there are `2**n_subjects` number of possible permutations, while in the the two-sample test, there are `n_subjects!` number of possible permutations. In both cases, if the requested number of permutations equals or exceeds the exhaustive list of permutations, an exact test is performed using all possible permutations. However, in most cases the number of subjects will yield an a prohibitively large number of permutations, in which case a Monte Carlo approximate permutation test is used instead of an exact test.

In [0]:
# Compute ISCs and then run one-sample permutation test on ISCs
iscs = isc(data, pairwise=True, summary_statistic=None)
observed, p, distribution = permutation_isc(iscs, pairwise=True,
                                            summary_statistic='median',
                                            n_permutations=200)

In [0]:
# Inspect shape of null distribution
print(f"Null distribution shape = {distribution.shape}"
      f"\ni.e., {distribution.shape[0]} permutations "
      f"and {distribution.shape[1]} voxels")

# Get actual ISC value and p-value for first voxel
print(f"Actual observed ISC value for first voxel = {observed[0][0]:.3f},"
      f"\np-value from permutation test = {p[0]:.3f}")

In [0]:
# Note that with few subjects, an exact test is performed
data_n6 = data[:6]
iscs = isc(data_n6, pairwise=True, summary_statistic=None)
observed, p, distribution = permutation_isc(iscs, pairwise=True,
                                            summary_statistic='median',
                                            n_permutations=200)

If we have two groups we expect to have different ISC values, we must supply a `group_assignment` list. In the case of two groups, we compute the difference between the `summary_statistic` for each group. In the pairwise approach, we compute differences between the `summary_statistic` for within-group correlations, and ignore the between group correlations in the full subject-by-subject correlation matrix containing both groups. Furthermore, permutations are applied to subjects (i.e., rows and columns in the subject-by-subject correlation matrix) and not to pairs (which would disrupt the correlation structure among pairs). We'll construct a dataset where one group of subjects are noisier than the others.

In [0]:
# Create data with noisy subset of subjects
noisy_data = np.dstack((np.dstack((
    simulated_timeseries(n_subjects // 2, n_TRs,
                         n_voxels=n_voxels, noise=1))),
                        np.dstack((
    simulated_timeseries(n_subjects // 2, n_TRs,
                         n_voxels=n_voxels, noise=5)))))

# Create group_assignment variable with group labels
group_assignment = [1]*10 + [2]*10
print(f"Group assignments: \n{group_assignment}")

# Compute ISCs and then run two-sample permutation test on ISCs
iscs = isc(noisy_data, pairwise=True, summary_statistic=None)
observed, p, distribution = permutation_isc(iscs,
                                            group_assignment=group_assignment,
                                            pairwise=True,
                                            summary_statistic='median',
                                            n_permutations=200)

In [0]:
# Inspect shape of null distribution
print(f"Null distribution shape = {distribution.shape}"
      f"\ni.e., {distribution.shape[0]} permutations "
      f"and {distribution.shape[1]} voxels")

# Get actual ISC value and p-value for first voxel
print(f"Actual observed group difference in ISC values "
      f"for first voxel = {observed[0]:.3f},"
      f"\np-value from permutation test = {p[0]:.3f}")

## Correcting for multiple tests
Evaluating the statistical significance of an ISC analysis across many voxels will result in many false positives unless we somehow control for the large number of statistical tests. Here we'll use two simple approaches for correcting for multiple tests. In the first approach, we'll account for multiple tests by controlling the expected proportion of false positives or false discovery rate (FDR; [Benjamini & Hochberg, 1995](https://www.jstor.org/stable/2346101); [Benjamini & Yekutieli, 2001](https://www.jstor.org/stable/2674075); [Genovese et al., 2002](https://doi.org/10.1006/nimg.2001.1037)). In the second approach, we'll control the family-wise error rate (FWER) by constructing a null distribution from the maximum ISC value across all voxels at each iteration of a randomization test ([Nichols & Holmes, 2002](https://doi.org/10.1002/hbm.1058)).

First, we'll create a dataset where half the voxels are very consistent across subjects and the other half are very noisy.

In [0]:
# Create data with where half of voxels are noisy
noisy_data = np.hstack((np.dstack((
    simulated_timeseries(n_subjects, n_TRs,
                         n_voxels=n_voxels // 2, noise=1))),
                        np.dstack((
    simulated_timeseries(n_subjects, n_TRs,
                         n_voxels=n_voxels // 2, noise=9)))))

# Visualize data for first subject where half of voxels are noisy
plt.matshow(noisy_data[..., 0], cmap='RdYlBu_r', vmin=-3, vmax=3)
plt.grid(False)
plt.xlabel('voxels')
plt.ylabel('time points');

After visualizing these data, we'll compute ISCs and use a one-sample two-sided bootstrap hypothesis test, which yields *p*-values and a null distribution. For this example, we'll use a more realistic number of bootstrap samples (1,000)—this may take a couple minutes.

In [0]:
# Compute ISCs and then run bootstrap hypothesis test on ISCs
# using a realistic number of permutations (takes a few minutes)
iscs = isc(noisy_data, pairwise=True, summary_statistic=None)
observed, ci, p, distribution = bootstrap_isc(iscs, pairwise=True,
                                              ci_percentile=95,
                                              summary_statistic='median',
                                              n_bootstraps=1000)

### Controlling FDR
To control FDR, we'll use the the `multipletests` function from the StatsModels Python package. This returns an array of *q*-values, which are typically interpreted as FDR-corrected *p*-values. By thresholding uncorrected and corrected *p*- and *q*-values, we can determine how many voxels survived correction for multiple tests.

In [0]:
# Get q-values (i.e., FDR-controlled p-values) using statsmodels
q = multipletests(p, method='fdr_by')[1]

# We can also convert these q-values to z-values
z = np.abs(norm.ppf(q))

# Also get significant voxels with and without correction
corrected = q[np.newaxis, :] < .05
uncorrected = p[np.newaxis, :] < .05

# Count significant voxels before and after correction
print(f'{np.sum(uncorrected)} "significant" voxels before correction for '
      f"multiple tests; {np.sum(corrected)} significant voxels after "
      f"controlling FDR at .05")

Finally, we can visualize the voxel time series for an example subject, the ISC values across subjects, and which voxels are considered significant before and after controlling FDR at .05. Note that before correction even some of the noisy voxels are considered to have significant ISC; however, after correction, the number of significant noisy voxels is reduced.

In [0]:
# Set up grid of subplots for visualizing voxel values and significance
fig, (ax0, ax1, ax2, ax3) = plt.subplots(nrows=4, figsize=(12, 8),
                                         sharex=True,
                                         gridspec_kw={'height_ratios':
                                                      [300, 190, 20, 20]})

# Visualize data for first subject where half of voxels are noisy
ax0.matshow(noisy_data[..., 0], cmap='RdYlBu_r', vmin=-3, vmax=3)
ax0.grid(False)
ax0.set_ylabel('time points')
ax0.set_title('response time series for example subject', y=1)

# Visualize ISC values across all pairs of subjects
ax1.matshow(iscs, cmap='RdYlBu_r', vmin=-1, vmax=1)
ax1.grid(False)
ax1.set_ylabel('pairs of subjects')
ax1.set_title('ISC values for all pairs of subjects', y=1)

# Visualize uncorrected and corrected significant voxels
ax2.matshow(np.repeat(uncorrected, 20, axis=0),
            cmap='viridis',vmin=0, vmax=1)
ax2.grid(False)
ax2.set_yticks([])
ax2.set_title('uncorrrected "significant" voxels (yellow)')

ax3.matshow(np.repeat(corrected, 20, axis=0),
            cmap='viridis',vmin=0, vmax=1)
ax3.grid(False)
ax3.set_xlabel('voxels')
ax3.xaxis.tick_bottom()
ax3.set_yticks([])
ax3.set_title('FDR-corrrected significant voxels (yellow)')
plt.tight_layout()

### Controlling FWER
To strictly control the FWER, one method is to construct a null distribution of maximum ISC statistics across all voxels. First we'll use the `permutation_isc` function to run a one-sample two-sided permutation test using a sign-flipping procedure, which returns *p*-values and a null distribution.

In [0]:
# Compute ISCs and then run two-sample permutation test on ISCs
iscs = isc(noisy_data, pairwise=True, summary_statistic=None)
observed, p, distribution = permutation_isc(iscs, pairwise=True,
                                            summary_statistic='mean',
                                            n_permutations=1000)

Next, we'll write a simple function that takes a null distribution with multiple voxels, and aggregates the maximum ISC value across all voxels for each null sample.

In [0]:
# Loop through null distribution and get maximum value across voxels
def get_maxima(distribution):
  max_distribution = []
  for i in distribution:
    max_isc = np.amax(i)
    max_distribution.append(max_isc)
  max_distribution = np.array(max_distribution)
  return max_distribution

After we create a null distribution of maximum statistics, any voxel with an ISC value in the top 5% of distribution can be considered significant. Here we compute *p*-values from the null distribution of maximum statistics using a two-sided test.

In [0]:
# Create null distribution of maximum ISCs across all voxels
max_distribution = get_maxima(distribution)

# Broadcast our max distribution across all 1000 voxels
max_distribution = np.repeat(max_distribution[:, np.newaxis], 1000, axis=1)

# Get the summary statistic (median) for our actual ISC values
# since we set summary_statistic=None above
observed = np.median(iscs, axis=0)[np.newaxis, :]

# Evaluate whether observed ISCs land in the tail of the max distribution
p_max = ((np.sum(np.abs(max_distribution) >= np.abs(observed), axis=0) + 1) /
          float((len(max_distribution) + 1)))[np.newaxis, :]

# Get p-values less than .05 (corrected for multiple tests)
corrected = p_max < .05

As with the FDR approach,  we can visualize the data and the voxels marked as significant before and after correction for multiple tests. This method of correction for multiple tests is considerably more conservative.

In [0]:
# Set up grid of subplots for visualizing voxel values and significance
fig, (ax0, ax1, ax2, ax3) = plt.subplots(nrows=4, figsize=(12, 8),
                                         sharex=True,
                                         gridspec_kw={'height_ratios':
                                                      [300, 190, 20, 20]})

# Visualize data for first subject where half of voxels are noisy
ax0.matshow(noisy_data[..., 0], cmap='RdYlBu_r', vmin=-3, vmax=3)
ax0.grid(False)
ax0.set_ylabel('time points')
ax0.set_title('response time series for example subject', y=1)

# Visualize ISC values across all pairs of subjects
ax1.matshow(iscs, cmap='RdYlBu_r', vmin=-1, vmax=1)
ax1.grid(False)
ax1.set_ylabel('pairs of subjects')
ax1.set_title('ISC values for all pairs of subjects', y=1)

# Visualize uncorrected and corrected significant voxels
ax2.matshow(np.repeat(uncorrected, 20, axis=0),
            cmap='viridis',vmin=0, vmax=1)
ax2.grid(False)
ax2.set_yticks([])
ax2.set_title('uncorrrected "significant" voxels (yellow)')

ax3.matshow(np.repeat(corrected, 20, axis=0),
            cmap='viridis',vmin=0, vmax=1)
ax3.grid(False)
ax3.set_xlabel('voxels')
ax3.xaxis.tick_bottom()
ax3.set_yticks([])
ax3.set_title('FWER-corrrected significant voxels (yellow)')
plt.tight_layout()

Note that there are many other ways to correct for multiple tests, such as using cluster-extent thresholding; but these methods are beyond the scope of this tutorial.

## ISFC analysis
Rather than computing ISCs for corresponding voxels across participants, we can instead compute ISCs between all voxels to measure functional integration (i.e., connectivity). This method is called intersubject functional correlation (ISFC) analysis ([Simony et al., 2016](https://doi.org/10.1038/ncomms12141)). Using the `vectorize_isfcs` option, we can either return a tuple containing the condensed off-diagonal ISFC values and the diagonal ISC values or the square (redundant) ISFC values. If `vectorize_isfcs=True` (the default), the first array in the tuple contains the off-diagonal ISFC values for each pair of voxels as condensed by `scipy.spatial.distance.squareform` and is shaped `n_subjects` (or `n_pairs`) by `n_connections` where
```
n_connections = n_voxels * (n_voxels - 1) / 2
```
The second array in the tuple is the diagonal values shaped `n_subjects` (or `n_pairs`) by `n_voxels`. If `vectorize_isfcs=False`, we get a 3-dimensional array containing the square (redundant) ISFC and ISC values, shaped `n_subjects` (or `n_pairs`) by `n_voxels` by `n_voxels`.  If a `summary_statistic` is supplied, or only two subjects are input, the singleton first dimension is removed.

In [0]:
# Compute ISFCs using leave-one-out approach
isfcs, iscs = isfc(data, pairwise=False, vectorize_isfcs=True)

# Check shape of output ISFC values
print(f"ISFC output shape = {isfcs.shape}\ni.e., {isfcs.shape[0]} "
      f"left-out subjects by {isfcs.shape[1]} connections (i.e., voxel pairs)"
      f"\nISCs output shape = {iscs.shape}\ni.e., {iscs.shape[0]} "
      f"left-out subjects by {iscs.shape[1]} voxels")

Alternatively, we can retain the (redundant) structure of the ISFC matrices using `vectorize_isfcs=False` to yield a 3-dimensional array of shape `n_subjects` by `n_voxels` by `n_voxels`:

In [0]:
# Compute ISFCs using leave-one-out approach
isfcs = isfc(data, pairwise=False, vectorize_isfcs=False)

# Check shape of output ISFC values
print(f"ISFC output shape = {isfcs.shape}\ni.e., {isfcs.shape[0]} "
      f"left-out subjects by {isfcs.shape[1]} voxels by {isfcs.shape[2]} "
      "voxels")

We can also supply a `summary_statistic` to collapse the ISFC values over left-out subjects or pairs of subjects:

In [0]:
# Compute ISFCs using leave-one-out approach with mean
isfcs, iscs = isfc(data, pairwise=False, summary_statistic='mean',
                   vectorize_isfcs=True)

# Check shape of output ISFC values
print(f"Mean ISFC output shape = {isfcs.shape}\ni.e., {isfcs.shape[0]} "
      f"connections (i.e., voxel pairs)"
      f"\nMean ISC output shape = {iscs.shape}\ni.e., {iscs.shape[0]} "
      "voxels")

We can use the `brainiak.isc.squareform_isfc` convenience function to convert between the condensed representation of ISFCs (with ISCs) and the square (redundant) representation of ISFCs. This function mimics `scipy.spatial.distance.squareform`, but retains the diagonal ISC values.

In [0]:
from brainiak.isc import squareform_isfc

# Start with square (redundant) ISFCs and check shape
isfcs_sq = isfc(data, pairwise=False, vectorize_isfcs=False)
print(f"Square (redundant) ISFCs shape: {isfcs_sq.shape}")

# Convert these directly to condensed ISFCs (and ISCs)
isfcs_c, iscs = squareform_isfc(isfcs_sq)
print(f"Condensed ISFCs shape: {isfcs_c.shape}, "
      f"ISCs shape: {iscs.shape}")

# Convert these directly back to redundant ISFCs
isfcs_r = squareform_isfc(isfcs_c, iscs)
print(f"Converted redundant ISFCs shape: {isfcs_r.shape}")

# Check that they are identical to the original square ISFCs
assert np.array_equal(isfcs_sq, isfcs_r)

Let's confirm that the diagonal of the ISFC matrix represents each voxel correlated with itself across subjects—the conventional ISC described above. We can see that the conventional ISC analysis is in fact a subset of the ISFC analysis.

In [0]:
# Get ISC values directly from ISFC matrix
isfcs, iscs = isfc(data, pairwise=False, vectorize_isfcs=True)

# Check that these are the same as conventional ISCs
assert np.allclose(iscs, isc(data))

Finally, we can visualize the matrix of mean (or median) ISFC values. If we used `vectorize_isfcs=True`, we'll first need to apply `squareform_isfc` the ISFC (and ISC) values. The diagonal blocks represent the 10 artificial "networks" in our simulated data; the 100 voxels in each network are highly correlated with each other and largely uncorrelated with voxels in other networks.

In [0]:
# Recompute mean ISFCs
isfcs, iscs = isfc(data, pairwise=False, summary_statistic='mean',
                   vectorize_isfcs=True)

# Convert these to a square representation
isfcs = squareform_isfc(isfcs, iscs)

# Visual mean ISFC matrix
plt.matshow(isfcs, cmap="RdYlBu_r", vmin=-1, vmax=1)
plt.grid(False)
plt.xticks(np.arange(0, 1001, 100)[1:], np.arange(100, 1001, 100),
           rotation=45)
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')
plt.yticks(np.arange(0, 1001, 100)[1:], np.arange(100, 1001, 100))
plt.xlabel('voxels')
plt.ylabel('voxels')
ax = plt.gca()
plt.colorbar(fraction=0.046, pad=0.04);

We can inject some structure into our simulated data to yield a more realistic ISFC matrix.

In [0]:
# Create more structured simulated data with 7 "networks";
# don't worry about the details
from scipy.ndimage import gaussian_filter1d

def structured_timeseries(n_subjects, n_TRs, n_voxels=1000, noise=1):
  signals = np.random.randn(n_TRs, 3)
  networks = np.column_stack((signals + np.random.randn(n_TRs, 3) * noise,
                              signals[:, 0] + np.random.randn(n_TRs) * noise,
                              signals[:, 0] + np.random.randn(n_TRs) * noise,
                              -signals[:, 2] + np.random.randn(n_TRs) * noise,
                              signals[:, 2] + np.random.randn(n_TRs) * noise))
  networks = networks[:, [0, 3, 4, 5, 1, 2, 6]]
  six = np.random.randint(n_voxels // 20, n_voxels // 6, 6)
  seven = np.append(six, (n_voxels - np.sum(six)))
  voxels = np.column_stack([np.tile(network[:, np.newaxis], (1, extent))
                            for network, extent in zip(networks.T, seven)])
  areas = [0] + sorted(np.random.randint(0, 1000, 16))
  areas = np.diff(areas).tolist() + [(1000 - areas[-1])]
  noise_sources = np.random.randn(n_TRs, 7)
  structured_noise = np.column_stack([np.tile(
      (noise_sources[:, np.random.choice(range(7))] *
       np.random.choice([-1, 1, 1, 1]))[:, np.newaxis],
                                              (1, extent)) 
                                      for extent in areas])
  voxels = gaussian_filter1d(voxels, 8.0, axis=0)
  structured_noise = gaussian_filter1d(structured_noise, 8.0, axis=0)
  data = []
  for s in np.arange(n_subjects):
    data.append(voxels + structured_noise * noise * .2 +
                np.random.randn(n_TRs, n_voxels) * noise * 1.35)

  data = np.dstack(data)
  return data

structured_data = structured_timeseries(n_subjects, n_TRs)

Now, we can recompute mean ISFCs using the leave-one-out approach and visualize the resulting ISFC matrix.

In [0]:
# Compute ISFCs using leave-one-out approach with mean
isfcs, iscs = isfc(structured_data, pairwise=False, summary_statistic='mean',
                   vectorize_isfcs=True)

# Convert these to a square representation
isfcs = squareform_isfc(isfcs, iscs)

# Visual mean ISFC matrix
plt.matshow(isfcs, cmap="RdYlBu_r", vmin=-.3, vmax=.3)
plt.grid(False)
plt.xticks(np.arange(0, 1001, 100)[1:], np.arange(100, 1001, 100),
           rotation=45)
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')
plt.yticks(np.arange(0, 1001, 100)[1:], np.arange(100, 1001, 100))
plt.xlabel('voxels')
plt.ylabel('voxels')
ax = plt.gca()
plt.colorbar(fraction=0.046, pad=0.04);

## Real fMRI data
Next, we'll  download a publicly available fMRI dataset and run an ISC analysis. This dataset comprises fMRI data for 20 subjects listening to the spoken story [Pie Man](https://themoth.org/stories/pie-man) by Jim O'Grady (archived on the [Princeton DataSpace](https://dataspace.princeton.edu/jspui/handle/88435/dsp01dz010s83s)). Note that we use 20 subjects to minimize computational demands for this tutorial and recommend larger sample sizes for publication. The gzipped data archive file is ~1.5 GB in size, and may take a couple minutes to download and unzip. The functional data were acquired with 3 x 3 x 4 mm voxels and 1.5 s TRs. Data were preprocessed using [fMRIPrep](https://fmriprep.readthedocs.io/en/stable/) ([Esteban et al., 2018](https://doi.org/10.1038/s41592-018-0235-4)), including spatial normalization to MNI space (the T1-weighted [ICBM 2009c Nonlinear Asymmetric template](http://nist.mni.mcgill.ca/?p=904)). The data were then smoothed to 6 mm FWHM using [AFNI](https://afni.nimh.nih.gov/)'s [3dBlurToFWHM](https://afni.nimh.nih.gov/pub/dist/doc/program_help/3dBlurToFWHM.html) ([Cox, 1996](https://doi.org/10.1006/cbmr.1996.0014)). The following confound variables were regressed out using [3dTproject](https://afni.nimh.nih.gov/pub/dist/doc/program_help/3dTproject.html): six head motion parameters (and their first derivatives), framewise displacement, six prinicipal components from an anatomical mask of cerebrospinal fluid (CSF) and white matter, sine/cosine bases for high-pass filtering (cutoff: 0.00714 Hz; 140 s), as well as a linear and quadratic trends. The anatomical template and a brain mask (i.e., excluding skull) are supplied as well. These have been resampled to match resolution of the functional images.

In [0]:
# Download data tarball from Princeton DataSpace
from urllib.request import urlretrieve
urlretrieve('https://dataspace.princeton.edu/jspui/bitstream/'
            '88435/dsp01dz010s83s/6/pieman-isc-tutorial.tgz',
            'pieman-isc-tutorial.tgz');
!tar -xvzf pieman-isc-tutorial.tgz

### Loading MRI data
We'll use NiBabel as well as BrainIAK's `io` and `image` functionality to load the functional data and apply a brain mask.

In [0]:
# Import functions helpful for managing file paths
from glob import glob
from os.path import join

data_dir = 'pieman-isc-tutorial'

# Filenames for MRI data; gzipped NIfTI images (.nii.gz)
func_fns = glob(join(data_dir, ('sub-*_task-pieman_space-MNI152NLin2009cAsym'
                                '_desc-tproject_bold.nii.gz')))
mask_fn = join(data_dir, 'MNI152NLin2009cAsym_desc-brain_mask.nii.gz')
mni_fn = join(data_dir, 'MNI152NLin2009cAsym_desc-brain_T1w.nii.gz')

# Load a NIfTI of the brain mask as a reference Nifti1Image
ref_nii = nib.load(mask_fn)

# Load functional images and masks using brainiak.io
func_imgs = load_images(func_fns)
mask_img = load_boolean_mask(mask_fn)

# Get coordinates of mask voxels in original image
mask_coords = np.where(mask_img)

# Apply the brain mask using brainiak.image
masked_imgs = mask_images(func_imgs, mask_img)

# Collate data into a single TR x voxel x subject array
orig_data = MaskedMultiSubjectData.from_masked_images(masked_imgs,
                                                      len(func_fns))

The data from each subject are stacked along the third dimension, yielding a `n_TRs` by `n_voxels` by `n_subjects` array. The functional acquisition originally included 13 s of music and 2 s of silence prepended to the story stimulus and an additional 13 s of silence after the story (450 s or 300 TRs in total). These segments as well as the first 12 s (8 TRs) after story onset can be discarded to minimize stimulus onset/offset effects. We may also opt to z-score the time series for each voxel.

In [0]:
print(f"Original fMRI data shape: {orig_data.shape} "
      f"\ni.e., {orig_data.shape[0]} time points, {orig_data.shape[1]} voxels, "
      f"{orig_data.shape[2]} subjects")

# Trim off non-story TRs and 12 s post-onset
data = orig_data[18:-8, ...]

print(f"Trimmed fMRI data shape: {data.shape} "
      f"\ni.e., {data.shape[0]} time points, {data.shape[1]} voxels, "
      f"{data.shape[2]} subjects")

# Z-score time series for each voxel
data = zscore(data, axis=0)

### ISC analysis
Next, we'll run a leave-one-out ISC analysis on the preprocessed fMRI data, including all voxels in the brain mask—this may take a few minutes. Note that some voxels with no variance over time for one or more subjects were included in the brain mask due to the limited field of view during EPI acquisition and susceptibility artifacts (signal dropout). This yields NaN (not a number) values. Here, we'll use set `tolerate_nans` to `0.8` to ensure that, when computing the averaging time series for *N*–1 subjects in the leave-one-out approach, only voxels with >= 80% of subjects have non-NaN values are included.

In [0]:
# Leave-one-out approach
iscs = isc(data, pairwise=False, tolerate_nans=.8)


# Check shape of output ISC values
print(f"ISC values shape = {iscs.shape} \ni.e., {iscs.shape[0]} "
      f"left-out subjects and {iscs.shape[1]} voxel(s)")

Since we didn't supply a `summary_statistic` in the `isc` call, we get an ISC value for each left-out subject (we'll need ISCs for each subject in the subsequent statistical test). If we want to preliminarily inspect the mean (or median) ISCs, we can apply the `brainiak.isc.compute_summary_statistic` function afterward. Note that if we specifed a `summary_statistic` in the `isc` call, the `isc` function would simply use `compute_summary_statistic` internally.

In [0]:
# Compute mean ISC (with Fisher transformation)
mean_iscs = compute_summary_statistic(iscs, summary_statistic='mean', axis=0)

print(f"ISC values shape = {mean_iscs.shape} \ni.e., {mean_iscs.shape[0]} "
      f"mean value across left-out subjects and {iscs.shape[1]} voxel(s)"
      f"\nMinimum mean ISC across voxels = {np.nanmin(mean_iscs):.3f}; "
      f"maximum mean ISC across voxels = {np.nanmax(mean_iscs):.3f}")


# Compute median ISC
median_iscs = compute_summary_statistic(iscs, summary_statistic='median',
                                        axis=0)

print(f"ISC values shape = {median_iscs.shape} \ni.e., {median_iscs.shape[0]} "
      f"median value across left-out subjects and {iscs.shape[1]} voxel(s)"
      f"\nMinimum median ISC across voxels = {np.nanmin(median_iscs):.3f}; "
      f"maximum median ISC across voxels = {np.nanmax(median_iscs):.3f}")

### Statistical testing
To test whether the observed ISCs are significantly greater than zero, we'll perform a bootstrap hypothesis test ([Chen et al., 2016](https://doi.org/10.1016/j.neuroimage.2016.05.023)). This may also take a couple minutes.

In [0]:
# Run bootstrap hypothesis test on ISCs
observed, ci, p, distribution = bootstrap_isc(iscs, pairwise=False,
                                              ci_percentile=95,
                                              summary_statistic='median',
                                              n_bootstraps=1000)

Before we correct for multiple tests, we should exclude any voxels with NaNs. To do this, we'll extract the non-NaN voxels, run the correction for multiple tests, then reinsert the non-NaN voxels into the full mask.

In [0]:
# Get number of NaN voxels
n_nans = np.sum(np.isnan(observed))
print(f"{n_nans} voxels out of {observed.shape[0]} are NaNs "
      f"({n_nans / observed.shape[0] * 100:.2f}%)")

# Get voxels without NaNs
nonnan_mask = ~np.isnan(observed)
nonnan_coords = np.where(nonnan_mask)

# Mask both the ISC and p-value map to exclude NaNs
nonnan_isc = observed[nonnan_mask]
nonnan_p = p[nonnan_mask]

Now we'll apply the `multipletests` function from StatsModels to the *p*-values from the bootstrap hypothesis test to control the false discovery rate (FDR) at 0.05 across all voxels. This yields a map of *q*-values. We can then threshold our ISC image based on the FDR-adjusted *q*-values, which are derived from the entire image, rather than the uncorrected *p*-values.

In [0]:
# Get FDR-controlled q-values
nonnan_q = multipletests(nonnan_p, method='fdr_by')[1]
threshold = .05
print(f"{np.sum(nonnan_q < threshold)} significant voxels "
      f"controlling FDR at {threshold}")

# Threshold ISCs according FDR-controlled threshold
nonnan_isc[nonnan_q >= threshold] = np.nan

# Reinsert thresholded ISCs back into whole brain image
isc_thresh = np.full(observed.shape, np.nan)
isc_thresh[nonnan_coords] = nonnan_isc

### Visualizing results
Finally, to visualize the significant ISC values, we must first reformat the 2-dimensional masked array into a 3-dimensional NIfTI image. We'll use an arbitrary reference NIfTI image `ref_nii` (the brain mask) to assign affine and header information correctly.

In [0]:
# Create empty 3D image and populate
# with thresholded ISC values
isc_img = np.full(ref_nii.shape, np.nan)
isc_img[mask_coords] = isc_thresh

# Convert to NIfTI image
isc_nii = nib.Nifti1Image(isc_img, ref_nii.affine, ref_nii.header)

We'll use `nilearn.plotting.plot_stat_map` to plot two views of the NIfTI image. We'll set the maximum ISC value for the colorbar at 0.5 and use a divergent colormap called `RdYlBu_r`. This yields maps where significant voxels are colored according to the median ISC value across left-out subjects. Statistical significance was assessed by a 
nonparametric bootstrap hypothesis test resampling left-out subjects and corrected for multiple tests by controlling FDR at .05.



In [0]:
# Install Nilearn for plotting functionality
!pip install nilearn
from nilearn.plotting import plot_stat_map
%matplotlib inline

In [0]:
# Plot slices at coordinates -61, -20, 8
plot_stat_map(
    isc_nii,
    cmap='RdYlBu_r',
    vmax=.5,
    cut_coords=(-61, -20, 8))

# Plot slices at coordinates 0, -65, 40
plot_stat_map(
    isc_nii,
    cmap='RdYlBu_r',
    vmax=.5,
    cut_coords=(0, -65, 40))
plt.show()

Some significant voxels have fairly low ISCs, so we can also use the `threshold` option in `plot_stat_map` to also exclude voxels with, e.g., ISC < .1.

In [0]:
# Plot slices at coordinates -61, -20, 8
plot_stat_map(
    isc_nii,
    cmap='RdYlBu_r',
    vmax=.5,
    threshold=.1,
    cut_coords=(-61, -20, 8))

# Plot slices at coordinates 0, -65, 40
plot_stat_map(
    isc_nii,
    cmap='RdYlBu_r',
    vmax=.5,
    threshold=.1,
    cut_coords=(0, -65, 40))

Note that here we are only analyzing responses to the fully intact Pie Man story, which yields high ISCs in both low-level auditory areas and higher-level areas processing narrative content. However, if we scrambled the presentation of the story so as to disrupt the temporally contiguous narrative, we would expect to see high ISCs only in low-level auditory areas ([Hasson et al., 2008](https://doi.org/10.1523/jneurosci.5487-07.2008); [Lerner et al., 2011](https://doi.org/10.1523/jneurosci.3684-10.2011)). To finish, we can also use `nib.save` to save the NIfTI image for use with other neuroimaging data analysis and visualization programs.

In [0]:
# Save final ISC NIfTI image as .nii
isc_fn = 'isc_thresh_pieman_n20.nii.gz'
nib.save(isc_nii, isc_fn)

## References and suggested reading

* Benjamini, Y., & Hochberg, Y. (1995). Controlling the false discovery rate: a practical and powerful approach to multiple testing. Journal of the Royal Statistical Society: Series B (Methodological), 289–300. https://www.jstor.org/stable/2346101

* Benjamini, Y., & Yekutieli, D. (2001). The control of the false discovery rate in multiple testing under dependency. *Annals of Statistics*, *29*(4), 1165–1188. https://www.jstor.org/stable/2674075

* Chen, G., Shin, Y. W., Taylor, P. A., Glen, D. R., Reynolds, R. C., Israel, R. B., & Cox, R. W. (2016). Untangling the relatedness among correlations, part I: nonparametric approaches to inter-subject correlation analysis at the group level. *NeuroImage*, *142*, 248–259. https://doi.org/10.1016/j.neuroimage.2016.05.023

* Chen, G., Taylor, P. A., Shin, Y. W., Reynolds, R. C., & Cox, R. W. (2017). Untangling the relatedness among correlations, part II: inter-subject correlation group analysis through linear mixed-effects modeling. *NeuroImage*, *147*, 825–840. https://doi.org/10.1016/j.neuroimage.2016.08.029

* Cox, R. W. (1996). AFNI: software for analysis and visualization of functional magnetic resonance neuroimages. *Computers and Biomedical Research*, *29*(3), 162–173. https://doi.org/10.1006/cbmr.1996.0014

* Esteban, O., Markiewicz, C., Blair, R. W., Moodie, C., Isik, A. I., Erramuzpe, A., Kent, J. D., Goncalves, M., DuPre, E., Snyder, M., Oya, H., Ghosh, S., Wright, J., Durnez, J., Poldrack, R., & Gorgolewski, K. J. (2018). fMRIPrep: a robust preprocessing pipeline for functional MRI. *Nature Methods*. https://doi.org/10.1038/s41592-018-0235-4

* Genovese, C. R., Lazar, N. A., & Nichols, T. (2002). Thresholding of statistical maps in functional neuroimaging using the false discovery rate. *NeuroImage*, *15*(4), 870–878. https://doi.org/10.1006/nimg.2001.1037

* Hasson, U., Ghazanfar, A. A., Galantucci, B., Garrod, S., & Keysers, C. (2012). Brain-to-brain coupling: a mechanism for creating and sharing a social world. *Trends in Cognitive Sciences*, *16*(2), 114–121. https://doi.org/10.1016/j.tics.2011.12.007

* Hasson, U., Malach, R., & Heeger, D. J. (2010). Reliability of cortical activity during natural stimulation. *Trends in Cognitive Sciences*, *14*(1), 40–48. https://doi.org/10.1016/j.tics.2009.10.011

* Hasson, U., Nir, Y., Levy, I., Fuhrmann, G., & Malach, R. (2004). Intersubject synchronization of cortical activity during natural vision. *Science*, *303*(5664), 1634–1640. https://doi.org/10.1126/science.1089506

* Hasson, U., Yang, E., Vallines, I., Heeger, D. J., & Rubin, N. (2008). A hierarchy of temporal receptive windows in human cortex. *Journal of Neuroscience*, *28*(10), 2539–2550. https://doi.org/10.1523/jneurosci.5487-07.2008

* Kauppi, J. P., Pajula, J., & Tohka, J. (2014). A versatile software package for inter-subject correlation based analyses of fMRI. *Frontiers in Neuroinformatics*, *8*, 2. https://doi.org/10.3389/fninf.2014.00002

* Lerner, Y., Honey, C. J., Silbert, L. J., & Hasson, U. (2011). Topographic mapping of a hierarchy of temporal receptive windows using a narrated story. *Journal of Neuroscience*, *31*(8), 2906–2915. https://doi.org/10.1523/jneurosci.3684-10.2011

* Nichols, T. E., & Holmes, A. P. (2002). Nonparametric permutation tests for functional neuroimaging: a primer with examples. *Human Brain Mapping*, *15*(1), 1–25. https://doi.org/10.1002/hbm.1058

* Silbert, L. J., Honey, C. J., Simony, E., Poeppel, D., & Hasson, U. (2014). Coupled neural systems underlie the production and comprehension of naturalistic narrative speech. *Proceedings of the National Academy of Sciences of the United States of America*, *111*(43), E4687–E4696. https://doi.org/10.1073/pnas.1323812111

* Simony, E., Honey, C. J., Chen, J., Lositsky, O., Yeshurun, Y., Wiesel, A., & Hasson, U. (2016). Dynamic reconfiguration of the default mode network during narrative comprehension. *Nature Communications*, *7*, 12141. https://doi.org/10.1038/ncomms12141

* Stephens, G. J., Silbert, L. J., & Hasson, U. (2010). Speaker–listener neural coupling underlies successful communication. *Proceedings of the National Academy of Sciences of the United States of America*, *107*(32), 14425–14430. https://doi.org/10.1073/pnas.1008662107